<a href="https://colab.research.google.com/github/nicodecoker/Transformers-Tutorials/blob/master/Fine_tuning_LayoutLMForSequenceClassification_on_RVL_CDIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we are going to fine-tune `LayoutLMForSequenceClassification` on the [RVL-CDIP dataset](https://www.cs.cmu.edu/~aharley/rvl-cdip/), which is a document image classification task. Each scanned document in the dataset belongs to one of 16 classes, such as "resume" or "invoice" (so it's a multiclass classification problem). The entire dataset consists of no less than 400,000 (!) scanned documents.

For demonstration purposes, we are going to fine-tune the model on a really small subset (one example per class), and verify whether the model is able to overfit them. Note that LayoutLM achieves state-of-the-art results on RVL-CDIP, with a classification accuracy of 94.42% on the test set.

* Original LayoutLM paper: https://arxiv.org/abs/1912.13318
* LayoutLM docs in the Transformers library: https://huggingface.co/transformers/model_doc/layoutlm.html


## Setting up environment

First, we install the 🤗 transformers and datasets libraries, as well as the [Tesseract OCR engine](https://github.com/tesseract-ocr/tesseract) (built by Google). LayoutLM requires an external OCR engine of choice to turn a document into a list of words and bounding boxes.

In [ ]:
! pip install transformers datasets

## Getting the data

Next, we download a small subset of the RVL-CDIP dataset (which I prepared), containing 15 documents (one example per class). I omitted the "handwritten" class, because the OCR results were mediocre.

In [ ]:
from datasets import load_dataset
dataset = load_dataset("RIPS-Goog-23/RVL-CDIP", split="train[:1000]")
dataset.save_to_disk("datasets/rvl-cdip")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset.save_to_disk("/content/drive/MyDrive/datasets/rvl-cdip")

In [ ]:
from datasets import load_from_disk
dataset = load_from_disk("/content/drive/MyDrive/datasets/rvl-cdip")

## Preprocessing the data using 🤗 datasets

First, we convert the dataset into a Pandas dataframe, having 2 columns: `image_path` and `label`.

In [ ]:
dataset.features

In [ ]:
labels = list(set(dataset['label']))
idx2label = {v: k for v, k in enumerate(labels)}
label2idx = {k: v for v, k in enumerate(labels)}
label2idx

Next, we can turn the word-level 'words' and 'bbox' columns into token-level `input_ids`, `attention_mask`, `bbox` and `token_type_ids` using `LayoutLMTokenizer`.

In [ ]:
dataset[0]['bbox']

In [ ]:
def convert_bbox(batch):
  bbox_new = []
  for bbox in batch["bbox"]:
    bbox_new.append([bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]])
  return {"bbox": bbox_new}

dataset = dataset.map(convert_bbox, batched=False)

In [ ]:
dataset[0]['bbox']

In [ ]:
from ast import literal_eval
dataset = dataset.map(lambda sample: {'words': literal_eval(sample['text'])})
dataset.features

In [ ]:
from transformers import LayoutLMTokenizer
import torch

tokenizer = LayoutLMTokenizer.from_pretrained("microsoft/layoutlm-base-uncased")

def encode_example(example, max_seq_length=512, pad_token_box=[0, 0, 0, 0]):
  words = example['words']
  normalized_word_boxes = example['bbox']

  assert len(words) == len(normalized_word_boxes)

  token_boxes = []
  for word, box in zip(words, normalized_word_boxes):
      word_tokens = tokenizer.tokenize(word)
      token_boxes.extend([box] * len(word_tokens))

  # Truncation of token_boxes
  special_tokens_count = 2
  if len(token_boxes) > max_seq_length - special_tokens_count:
      token_boxes = token_boxes[: (max_seq_length - special_tokens_count)]

  # add bounding boxes of cls + sep tokens
  token_boxes = [[0, 0, 0, 0]] + token_boxes + [[1000, 1000, 1000, 1000]]

  encoding = tokenizer(' '.join(words), padding='max_length', truncation=True)
  # Padding of token_boxes up the bounding boxes to the sequence length.
  input_ids = tokenizer(' '.join(words), truncation=True)["input_ids"]
  padding_length = max_seq_length - len(input_ids)
  token_boxes += [pad_token_box] * padding_length
  encoding['bbox'] = token_boxes
  encoding['label'] = label2idx[example['label']]

  assert len(encoding['input_ids']) == max_seq_length
  assert len(encoding['attention_mask']) == max_seq_length
  assert len(encoding['token_type_ids']) == max_seq_length
  assert len(encoding['bbox']) == max_seq_length

  return encoding

In [ ]:
from datasets import Features, Sequence, ClassLabel, Value, Array2D

# we need to define the features ourselves as the bbox of LayoutLM are an extra feature
features = Features({
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'label': ClassLabel(names=labels),
    'image': Value(dtype='string'),
    'text': Value(dtype='string'),
    'words': Sequence(feature=Value(dtype='string')),
})

encoded_dataset = dataset.map(lambda example: encode_example(example),
                              features=features)

Finally, we set the format to PyTorch, as the LayoutLM implementation in the Transformers library is in PyTorch. We also specify which columns we are going to use.

In [ ]:
encoded_dataset.set_format(type='torch', columns=['input_ids', 'bbox', 'attention_mask', 'token_type_ids', 'label'])

In [ ]:
dataloader = torch.utils.data.DataLoader(encoded_dataset, batch_size=1, shuffle=True)
batch = next(iter(dataloader))
batch

Let's verify whether the input ids are created correctly by decoding them back to text:

In [ ]:
tokenizer.decode(batch['input_ids'][0].tolist())

In [ ]:
idx2label[batch['label'][0].item()]

## Define the model

Here we define the model, namely `LayoutLMForSequenceClassification`. We initialize it with the weights of the pre-trained base model (`LayoutLMModel`). The weights of the classification head are randomly initialized, and will be fine-tuned together with the weights of the base model on our tiny dataset. Once loaded, we move it to the GPU.



In [ ]:
from transformers import LayoutLMForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
print(f'Running on {device}')

model = LayoutLMForSequenceClassification.from_pretrained(
    "microsoft/layoutlm-base-uncased",
    num_labels=len(label2idx))
model.to(device)

## Train the model

Here we train the model in familiar PyTorch fashion. We use the Adam optimizer with weight decay fix (normally you can also specify which variables should have weight decay and which not + a learning rate scheduler, see [here](https://github.com/microsoft/unilm/blob/5d16c846bec56b6e88ec7de4fc3ceb7c803571a4/layoutlm/examples/classification/run_classification.py#L94) for how the authors of LayoutLM did this), and train for 30 epochs. If the model is able to overfit it, then it means there are no issues and we can train it on the entire dataset.

In [ ]:
from torch.optim import AdamW
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)

global_step = 0
num_train_epochs = 30
t_total = len(dataloader) * num_train_epochs # total number of training steps

#put the model in training mode
model.train()
for epoch in tqdm(range(num_train_epochs), position=0, leave=True):
  print("Epoch:", epoch)
  running_loss = 0.0
  correct = 0
  for batch in tqdm(dataloader, position=0, leave=True):
      input_ids = batch["input_ids"].to(device)
      bbox = batch["bbox"].to(device)
      attention_mask = batch["attention_mask"].to(device)
      token_type_ids = batch["token_type_ids"].to(device)
      labels = batch["label"].to(device)

      # forward pass
      outputs = model(input_ids=input_ids,
                      bbox=bbox,
                      attention_mask=attention_mask,
                      token_type_ids=token_type_ids,
                      labels=labels)
      loss = outputs.loss

      running_loss += loss.item()
      predictions = outputs.logits.argmax(-1)
      correct += (predictions == labels).float().sum()

      # backward pass to get the gradients
      loss.backward()

      # update
      optimizer.step()
      optimizer.zero_grad()
      global_step += 1

  print("Loss:", running_loss / batch["input_ids"].shape[0])
  accuracy = 100 * correct / len(data)
  print("Training accuracy:", accuracy.item())

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(output_dir="layoutlm-sequenceclassification-rvlcdip",
                         overwrite_output_dir=True,
                         remove_unused_columns=False,
                         warmup_steps=0.1,
                         max_steps=2000,
                         evaluation_strategy="steps",
                         eval_steps=100,
                         push_to_hub=False)

trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )